In [1]:
import pandas as pd 
import numpy as np 
import os
import torch
from torch import nn

# Deep Neural Networks Laboratory

## A first, shallow Neural Network

To check if the CUDA version of Pytorch was installed successfully, only if your machine supports CUDA, you can do:

In [2]:
torch.cuda.is_available()   

True

The input of a neural network will be composed of two tensors: the sample and the label.

Imagine we have a classification task with 3 possible labels (0,1,2) and samples of 5 features (5-dimensional samples).

We will have tensors like:

In [3]:
x = torch.rand(1,5)
y = torch.randint(3, (1,)).long()

In [4]:
x

tensor([[0.4045, 0.0240, 0.9723, 0.0044, 0.8372]])

In [5]:
y

tensor([0])

Let us define a shallow, very simple Neural Network.

The Neural Network will have: 
1. As many input neurons as sample features (5 in this case)
2. An hidden layer 
3. As many ouput neurons as possible labels (3 in this case)

![images/shallow_nn.png](images/shallow_nn.png) 

This schema entails 4 different operations:
1. Matrix multiplication with the weights of the connections from the input layer to the FC Layer
2. Activation function in the FC Layer
3. Matrix multiplication with the weights of the connections from the FC Layer to the output layer
4. Activation function in the output layer

In [6]:
class ShallowNeuralNetwork(nn.Module):
    def __init__(self, input_size, num_hidden, output_size):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.fc1 = nn.Linear(input_size, num_hidden)
        self.output_layer = nn.Linear(num_hidden, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.output_layer(x)
        x = self.softmax(x)
        return x

## Step by step computation of a forward pass

So, we start with the tensor x.

In [7]:
x

tensor([[0.4045, 0.0240, 0.9723, 0.0044, 0.8372]])

x goes through the hidden layer. What this means is that we obtain the output:

$ Wx + b $ where:
1. W is the weight matrix of the connections
2. b is the bias vector

These values depend on the chosen initialization and might be random at first.

In [8]:
hidden_layer = nn.Linear(5,10)

print( hidden_layer.weight)

print( hidden_layer.bias )

Parameter containing:
tensor([[-0.0070,  0.1379, -0.2972, -0.3857,  0.3767],
        [ 0.3181,  0.3841, -0.4351, -0.0183, -0.0194],
        [-0.2397, -0.0694,  0.4098,  0.4147,  0.2735],
        [-0.1449, -0.1749,  0.3753,  0.3086,  0.2791],
        [-0.0386,  0.0840, -0.4012, -0.1547,  0.2860],
        [-0.1574, -0.1003,  0.0241, -0.0021, -0.3871],
        [-0.2208,  0.1001,  0.0337,  0.2741, -0.2626],
        [-0.1746, -0.3350, -0.3427, -0.0808,  0.1582],
        [ 0.3688, -0.2545,  0.0900,  0.3000, -0.2573],
        [-0.0169, -0.4140,  0.3259,  0.1208,  0.3073]], requires_grad=True)
Parameter containing:
tensor([-0.2639,  0.0529,  0.3420, -0.4028,  0.2408,  0.2938, -0.1459,  0.2246,
        -0.0782, -0.3392], requires_grad=True)


In [9]:
w = hidden_layer.weight
b = hidden_layer.bias

output = torch.matmul(x, w.t()) + b
output

tensor([[-0.2387, -0.2485,  0.8727,  0.1342,  0.0759, -0.0730, -0.4187, -0.0552,
         -0.0617,  0.2186]], grad_fn=<AddBackward0>)

If we apply the hidden layer, we obtain the same results:

In [10]:
hidden_layer(x)

tensor([[-0.2387, -0.2485,  0.8727,  0.1342,  0.0759, -0.0730, -0.4187, -0.0552,
         -0.0617,  0.2186]], grad_fn=<AddmmBackward0>)

![images/dnn_2.jpeg](images/dnn_2.jpeg) 

In order to introduce non-linearity into our net, allowing it to learn more complex patterns, we pass the output of the FC layer through the ReLU activation function, which looks like this:


![images/relu.png](images/relu.png) 

In [11]:
output

tensor([[-0.2387, -0.2485,  0.8727,  0.1342,  0.0759, -0.0730, -0.4187, -0.0552,
         -0.0617,  0.2186]], grad_fn=<AddBackward0>)

In [12]:
relu = nn.ReLU()

output = relu(output)
output

tensor([[0.0000, 0.0000, 0.8727, 0.1342, 0.0759, 0.0000, 0.0000, 0.0000, 0.0000,
         0.2186]], grad_fn=<ReluBackward0>)

In [13]:
relu(hidden_layer(x))

tensor([[0.0000, 0.0000, 0.8727, 0.1342, 0.0759, 0.0000, 0.0000, 0.0000, 0.0000,
         0.2186]], grad_fn=<ReluBackward0>)

Now, we go through another round of linear multiplication from the nodes inside the hidden layer to the nodes in the output layer.

In [14]:
output_layer = nn.Linear(10,3)

weights_2 = output_layer.weight 
bias_2 = output_layer.bias

output = torch.matmul(output, weights_2.t()) + bias_2
output

tensor([[ 0.1568,  0.2774, -0.3305]], grad_fn=<AddBackward0>)

In [15]:
output_layer(relu(hidden_layer(x)))

tensor([[ 0.1568,  0.2774, -0.3305]], grad_fn=<AddmmBackward0>)

We have obtained a tensor of the same size of the output layer, which is the same size of the number of possible labels in our classification task.


In order to convert these values to probabilities, we apply the SoftMax activation function.

![images/softmax.png](images/softmax.png) 

In [16]:
softmax = nn.Softmax(dim=1)

output = softmax(output)
output

tensor([[0.3646, 0.4114, 0.2240]], grad_fn=<SoftmaxBackward0>)

In [17]:
softmax(output_layer(relu(hidden_layer(x))))

tensor([[0.3646, 0.4114, 0.2240]], grad_fn=<SoftmaxBackward0>)

The outputs are the probabilities of the sample being labeled with each one of the labels (0,1,2).

We manually implemented a forward pass of the network we defined at the start:

In [18]:
class ShallowNeuralNetwork(nn.Module):
    def __init__(self, input_size, num_hidden, output_size):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.fc1 = nn.Linear(input_size, num_hidden)
        self.output_layer = nn.Linear(num_hidden, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.output_layer(x)
        x = self.softmax(x)
        return x

In [19]:
shallow_nn = ShallowNeuralNetwork(5,10,3)

The values might be slightly different due to the randomization of the starting weights:

In [20]:
shallow_nn(x)

tensor([[0.2929, 0.3337, 0.3734]], grad_fn=<SoftmaxBackward0>)